In [37]:
import numpy as np
import matplotlib.pyplot as plt

import math

import sympy as sp

from sympy import latex
from IPython.display import display, Math

## Things which are given / can be calculated 

In [38]:
P_d_sp = sp.symbols('P_d')
D_p_sp, D_g_sp = sp.symbols('D_p D_g')
N_p_sp, N_g_sp, N_g_guess_sp = sp.symbols('N_p N_g, N_g_guess') 
C_sp, F_sp = sp.symbols('C F')
n_p_sp, n_g_sp, n_g_guess_sp = sp.symbols('n_p n_g n_g_guess')
VR_sp, VR_guess_sp, v_t_sp = sp.symbols('VR VR_guess v_t')
power_sp, lifetime_sp, Torque_sp = sp.symbols('power lifetime torque')
W_t_sp = sp.symbols('W_t')


#enter given values

uk = sp.nan
all_values = {
    P_d_sp: 12,
    D_p_sp: sp.nan,
    D_g_sp: uk,
    
    N_p_sp: 19,
    N_g_sp: uk,
    N_g_guess_sp: uk,
    n_p_sp: 1200,
    n_g_sp: uk,
    n_g_guess_sp: 387,
    VR_sp: sp.nan,
    VR_guess_sp: sp.nan,
    v_t_sp: sp.nan, 
    Torque_sp: uk,
    power_sp: 5
}

phi = 20 # pressure angle

lifetime = 15000
F_p = 1 # update if given
F_g = 1 # update if given

all_values, len(all_values)


({P_d: 12,
  D_p: nan,
  D_g: nan,
  N_p: 19,
  N_g: nan,
  N_g_guess: nan,
  n_p: 1200,
  n_g: nan,
  n_g_guess: 387,
  VR: nan,
  VR_guess: nan,
  v_t: nan,
  torque: nan,
  power: 5},
 14)

## Things to find from a table

some, like Y,Z can only be found later, tbd how to handle this

In [39]:
J_p = 0.34
J_G = 0.41

I = 0.1

C_p = 2300 #for steel

k_o = 1.5
k_s = 1.0

k_B = 1.0
k_v = 1.25
k_R = 1.0


#Need cycles to find, but these are not calculated until later
#im not sure the best way to handle this
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922


#for k_mb
cond = "2 straddle".upper()
# cond = "1 straddle".upper()
# cond = "0 straddle".upper()

safety_factor = 1


In [77]:
#k_mb
assert(cond is not None)

k_ma = None

if cond == "2 straddle".upper():
    k_mb = 1
elif cond == "1 straddle".upper():
    k_mb = 1.1
elif cond == "0 straddle".upper():
    k_mb = 1.25

assert(k_mb is not None)

k_mb

1.25

## Create the equations

In [40]:

eq1 = sp.Eq(VR_guess_sp, n_p_sp / n_g_guess_sp)
eq2 = sp.Eq(N_g_guess_sp, N_p_sp * VR_guess_sp)

eq3 = sp.Eq(VR_sp, N_g_sp / N_p_sp)

eq4 = sp.Eq(n_g_sp, n_p_sp / VR_sp)
eq5 = sp.Eq(D_p_sp, N_p_sp / P_d_sp)
eq6 = sp.Eq(D_g_sp, N_g_sp / P_d_sp)

eq7 = sp.Eq(v_t_sp, np.pi * D_p_sp * n_p_sp / 12)
eq8 = sp.Eq(VR_sp, n_p_sp / n_g_sp)

eq9 = sp.Eq(power_sp, Torque_sp * n_p_sp / 63000)


## Solve the equations 

In [41]:
#creates a thing of all the non-nan values
known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value


Solve the equations for VR_guess, n_g_guess, N_p, n_g

In [42]:
# Solve equations eq1 and eq2
solutions_eq1_eq2 = sp.solve([eq1, eq2], dict=True)

# Display the solutions
# for solution in solutions_eq1_eq2:
#     for var, expr in solution.items():
#         print(f"{var} = {expr}")


for solution in solutions_eq1_eq2:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values

{P_d: 12,
 N_p: 19,
 n_p: 1200,
 n_g_guess: 387,
 power: 5,
 N_g_guess: 7600/129,
 VR_guess: 400/129}

In [43]:

if known_values[N_g_guess_sp] == sp.nan:  
    a = sp.nan
else:
    known_values[N_g_sp] = round(known_values[N_g_guess_sp], 0)
# known_values[N_g_sp]

# Solve equation eq3 with the updated known values

if all_values[n_g_guess_sp] == sp.nan:
    pass

else:

    solution_eq3 = sp.solve(eq3.subs(known_values), dict=True)

    # Update known values with the solution of eq3
    for sol in solution_eq3:
        for var, expr in sol.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values




{P_d: 12,
 N_p: 19,
 n_p: 1200,
 n_g_guess: 387,
 power: 5,
 N_g_guess: 7600/129,
 VR_guess: 400/129,
 N_g: 59.0000000000000,
 VR: 3.10526315789474}

#### Solve the equations multiple times bc some of them depend on others

In [44]:
# Iteratively solve equations until all variables are found

run_count = 0
while True:

    previous_known_values = known_values.copy()
    
    # Solve equations with current known values
    solutions = sp.solve([eq4, eq5.subs(known_values), eq6.subs(known_values), eq7.subs(known_values), eq8.subs(known_values), eq9.subs(known_values)], dict=True)

    # Update known values with solutions
    print(len(solutions))
    for solution in solutions:

        for var, expr in solution.items():

            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)    

    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1
        print(f"Run interation: {run_count}")

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


1
Run interation: 1
0
P_d = 12
N_p = 19
n_p = 1200
n_g_guess = 387
power = 5
N_g_guess = 7600/129
VR_guess = 400/129
N_g = 59.0000000000000
VR = 3.10526315789474
D_g = 4.91666666666667
D_p = 1.58333333333333
n_g = 386.440677966102
torque = 262.500000000000
v_t = 497.418836818383


### Lists each variable and its equation 

#### Assign each variable a value, eventually these should all be numbers

In [45]:
# assert(len(known_values) == len(all_values))
len(known_values), len(all_values)
# known_values

for var, value in known_values.items():
    all_values[var] = value


P_d = all_values[P_d_sp]
D_p = all_values[D_p_sp]
D_g = all_values[D_g_sp]
N_p = all_values[N_p_sp]
N_g = all_values[N_g_sp]
n_p = all_values[n_p_sp]
n_g = all_values[n_g_sp]
n_g_guess = all_values[n_g_guess_sp]
VR = all_values[VR_sp]
VR_guess = all_values[VR_guess_sp]
v_t = all_values[v_t_sp]
power = all_values[power_sp]
Torque = all_values[Torque_sp]

P_d = float(P_d)
D_p = float(D_p)
D_g = float(D_g)
N_p = float(N_p)
N_g = float(N_g)
n_p = float(n_p)
n_g = float(n_g)
n_g_guess = float(n_g_guess)
VR = float(VR)
VR_guess = float(VR_guess)
vt = float(v_t)
power = float(power)
Torque = float(Torque)

all_values



{P_d: 12,
 D_p: 1.58333333333333,
 D_g: 4.91666666666667,
 N_p: 19,
 N_g: 59.0000000000000,
 N_g_guess: 7600/129,
 n_p: 1200,
 n_g: 386.440677966102,
 n_g_guess: 387,
 VR: 3.10526315789474,
 VR_guess: 400/129,
 v_t: 497.418836818383,
 torque: 262.500000000000,
 power: 5}

### Final assorted Calculations

### Find Cone angles

In [46]:
gamma = 180 / np.pi * np.arctan(N_p/N_g)

#gear cone angle

Gamma = 180 / np.pi * np.arctan(N_g/N_p)

assert(gamma + Gamma - 90 < 10**-8)


# Display the equations in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('gamma'), gamma))))
display(Math(latex(sp.Eq(sp.Symbol('Gamma'), Gamma))))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find Cone Distances

In [47]:
#outer cone distance

A_oG = D_g / (2 * np.sin(Gamma * np.pi / 180))

A_oP = D_p / (2 * np.sin(gamma * np.pi / 180))

A_oG, A_oP
display(Math(latex(sp.Eq(sp.Symbol('A_oG'), A_oG))))
display(Math(latex(sp.Eq(sp.Symbol('A_oP'), A_oP))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find r_m and R_m 

In [48]:
r_m = D_p / 2 - F_p / 2 * np.sin(gamma * np.pi / 180)

R_m = D_g / 2 - F_g / 2 * np.sin(Gamma * np.pi / 180)

r_m, R_m

# Display the equations in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('r_m'), r_m))))
display(Math(latex(sp.Eq(sp.Symbol('R_m'), R_m))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find Forces

In [49]:
W_t = Torque / r_m

wt2 = 33000 * power / v_t

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(W_t_sp, W_t)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

Radial Force

In [50]:
W_rp = W_t * np.tan(phi * np.pi / 180) * np.cos(gamma * np.pi / 180)
# Display the equation in LaTeX format with units of lbs


display(Math(latex(sp.Eq(sp.Symbol('W_rp'), W_rp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

Axial Force

In [59]:
W_xp = W_t * np.tan(phi * np.pi / 180) * np.sin(gamma * np.pi / 180)
display(Math(latex(sp.Eq(sp.Symbol('W_xp'), W_xp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

### Find Face Width

In [52]:
if F_p is not None:
    pass

else:

    F_p = 0.3 * A_oP #nominal face width
    F_g = 0.3 * A_oG #nominal face width

    if F_p > 10 / P_d:
        F_p = 10 / P_d
        user_in = input("face width at max")

    if F_g > 10 / P_d:
        F_g = 10 / P_d
        user_in = input("face width at max")


F_p, F_g
display(Math(latex(sp.Eq(sp.Symbol('F_p'), F_p))))
display(Math(latex(sp.Eq(sp.Symbol('F_g'), F_g))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Find

### Calculate Center Distance

In [53]:
C = (N_p + N_g) / (2 * P_d)
C
display(Math(latex(sp.Eq(C_sp, C))))

<IPython.core.display.Math object>

## Calulating correction Factors

### Calculate $C_{pf}$

only does _p not _g

In [54]:
#C_pf
C_pf = None

if F_p / D_p < 0.5:
    
    #___________Use the graph too find C_pf_____________________
    C_pf = 0

elif F_p <= 1.0:
    C_pf = F_p / (10 * D_p) - 0.025
elif F_p <= 15:
    C_pf = F_p / (10 * D_p) - 0.0375 + 0.0125 * F_p

assert(C_pf is not None)

C_pf
display(Math(latex(C_pf)))

<IPython.core.display.Math object>

### Calculate $C_{ma}$

In [55]:
#C_ma

assert(cond is not None)

C_ma = None

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F_p - 1.093 * 10**-4 * F_p**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F_p - 0.926 * 10**-4 * F_p**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F_p - 0.822 * 10**-4 * F_p**2

assert(C_ma is not None)

# Display the equation in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('C_ma'), C_ma))))

<IPython.core.display.Math object>

In [56]:
k_m = 1 + C_ma + C_pf

display(Math(latex(sp.Eq(sp.Symbol('k_m'), k_m))))

<IPython.core.display.Math object>

## Loading Cycles $N_{cp}, N_{cg}$

In [60]:
# Number of cycles

N_cp = 60 * lifetime * n_p
N_cg = 60 * lifetime * n_g

# Print in LaTeX with scientific notation
display(Math(r"N_{cp} = " + latex(N_cp) + r" \approx " + f"{N_cp:.2e}"))
display(Math(r"N_{cg} = " + latex(N_cg) + r" \approx " + f"{N_cg:.2e}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### *** Implement $Y_{xx}$  and $Z_{xx}$

Put all the graphs here

## Stresses and allowable Stresses

### $S_{tp}$, $S_{tG}$

In [63]:
s_tp = (W_t * P_d) / (F_p * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('s_tp'), s_tp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

In [62]:
s_tG = s_tp * J_p / J_G

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('s_tG'), s_tG)) + r" \text{ lbs}"))
s_tG

<IPython.core.display.Math object>

26645.79232691582

### $S_{atp}$, $S_{atG}$

In [67]:
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
display(Math(latex(sp.Eq(sp.Symbol('s_atP >'), s_atP_Greater_than)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

In [69]:
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng

display(Math(latex(sp.Eq(sp.Symbol('s_atG >'), s_atG_Greater_than)) + r" \text{ lbs}"))
s_atG_Greater_than

<IPython.core.display.Math object>

26645.79232691582

### Calc $S_c$

In [72]:

aa = (W_t * k_o * k_s * k_m  * k_v)

bb = (F_p * D_p * I)

S_c = C_p * np.sqrt( float( aa / bb)  ) #
# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('S_c'), S_c)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

### FInd adjusted values of $S_c$

In [75]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('S_acP >'), S_acP_Greater_than)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

In [76]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('S_acg >'), S_acg_Greater_than)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

## Brinell Hardness Calculations

In [ ]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

### use tables to find the appropriate Material, and enter its brinell hardness

In [168]:
HB = None

### Calculate actaul allowed bending and contact stresses

In [165]:
#too lazy to implement rn

S_acp = None
S_atp = None

S_acg = None
S_atg = None


if int(HB_grade) == 1:
    S_acp = (HB * 0.322 + 29.1 ) * 1000 
    S_atp = (HB * 0.0773 + 12.8 ) * 1000

    S_acg = (HB * 0.322 + 29.1 ) * 1000
    S_atg = (HB * 0.0773 + 12.8 ) * 1000

elif int(HB_grade) == 2:

    S_acp = (HB * 0.349 + 34.3 ) * 1000
    S_atp = (HB * 0.102 + 16.4 ) * 1000

    S_acg = (HB * 0.349 + 34.3 ) * 1000
    S_atg = (HB * 0.102 + 16.4 ) * 1000
    
else:
    assert(False)


### Calculate Safety factor

In [ ]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")



## Power transmitting Capacity

In [ ]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")